In [25]:
# libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
import tensorflow as tf
import numpy as np

In [26]:
# load the dataset
data = pd.read_csv("<--- DATASET LOCATION --->")
data.head()

,N,P,K,temperature,humidity,ph,moisture,label
0,90,42,43,20.879744,82.002744,6.502985,80,rice
1,85,58,41,21.770462,80.319644,7.038096,92,rice
2,60,55,44,23.004459,82.320763,7.840207,86,rice
3,74,35,40,26.491096,80.158363,6.980401,87,rice
4,78,42,42,20.130175,81.604873,7.628473,82,rice


In [27]:
# encode the target variable and save the corresponding labels
data["label"], LABELS = pd.factorize(data["label"])
print(LABELS)

Index(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas', 'mothbeans',
       'mungbean', 'blackgram', 'lentil', 'pomegranate', 'banana', 'mango',
       'grapes', 'watermelon', 'muskmelon', 'apple', 'orange', 'papaya',
       'coconut', 'cotton', 'jute', 'coffee'],
      dtype='object')


In [28]:
# separate features and target variable
X = data[["N", "P", "K", "temperature", "humidity", "ph", "moisture"]]
y = data["label"]
print(X.shape, y.shape)

(2200, 7) (2200,)


In [29]:
# split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
# normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [35]:
# ann model
model = models.Sequential()
model.add(layers.Dense(18, activation='relu', input_shape=(X_train.shape[1],))),
model.add(layers.Dense(9, activation='relu'))
model.add(layers.Dense(22, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 18)                144       
                                                                 
 dense_11 (Dense)            (None, 9)                 171       
                                                                 
 dense_12 (Dense)            (None, 22)                220       
                                                                 
Total params: 535 (2.09 KB)
Trainable params: 535 (2.09 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [37]:
# train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
44/44 [==============================] - 0s 3ms/step - loss: 3.0901 - accuracy: 0.0384 - val_loss: 3.0363 - val_accuracy: 0.0597
Epoch 2/50
44/44 [==============================] - 0s 801us/step - loss: 2.9955 - accuracy: 0.1009 - val_loss: 2.9628 - val_accuracy: 0.1108
Epoch 3/50
44/44 [==============================] - 0s 840us/step - loss: 2.8944 - accuracy: 0.2024 - val_loss: 2.8558 - val_accuracy: 0.2131
Epoch 4/50
44/44 [==============================] - 0s 849us/step - loss: 2.7456 - accuracy: 0.2578 - val_loss: 2.7044 - val_accuracy: 0.2131
Epoch 5/50
44/44 [==============================] - 0s 856us/step - loss: 2.5686 - accuracy: 0.2635 - val_loss: 2.5363 - val_accuracy: 0.2301
Epoch 6/50
44/44 [==============================] - 0s 810us/step - loss: 2.3829 - accuracy: 0.2777 - val_loss: 2.3526 - val_accuracy: 0.2443
Epoch 7/50
44/44 [==============================] - 0s 888us/step - loss: 2.1903 - accuracy: 0.3132 - val_loss: 2.1637 - val_accuracy: 0.2869
Epoch 8/

In [38]:
# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

14/14 [==============================] - 0s 503us/step - loss: 0.3045 - accuracy: 0.9023
Test Accuracy: 0.9022727012634277


In [39]:
# prediction function
def prediction(data):
    # normalization
    data = scaler.transform(data)
    # predict the crop label using the trained model
    prediction = model.predict(data)
    predicted_indx = np.argmax(prediction)
    predicted_crop = LABELS[predicted_indx]
    print("Predicted crop: ")
    print(predicted_crop)

In [40]:
prediction(np.array([[90,42,43,20.87974371,82.00274423,6.502985292000001,80]]))

1/1 [==============================] - 0s 50ms/step
Predicted crop: 
rice


/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [42]:
# convert the keras model to tensorflow lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# save it as .tflite
with open("/Users/ravi/Documents/crs/crs.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/pb/956wsyzs3y9128nmccpz4djc0000gn/T/tmpqmb1wns6/assets


INFO:tensorflow:Assets written to: /var/folders/pb/956wsyzs3y9128nmccpz4djc0000gn/T/tmpqmb1wns6/assets
2024-03-20 11:38:20.225393: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-03-20 11:38:20.225406: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 6, Total Ops 13, % non-converted = 46.15 %
 * 6 ARITH ops

- arith.constant:    6 occurrences  (f32: 6)



  (f32: 3)
  (f32: 1)


In [46]:
# print the findings
print(scaler.mean_, scaler.scale_)
print(model.weights)

[50.68636364 53.40965909 48.13636364 25.57963982 71.55999729  6.47593982
 73.20227273] [37.28732779 33.21872791 51.06361492  5.08862327 22.01775167  0.77322534
  8.44864133]
[<tf.Variable 'dense_10/kernel:0' shape=(7, 18) dtype=float32, numpy=
array([[-0.42171738,  0.34793925, -0.19519354,  0.83542615,  0.22893788,
         0.37455443, -0.657108  , -0.95058537,  0.8869077 , -0.29942748,
        -0.54381746, -0.91487074, -1.0931809 ,  0.916552  ,  0.5750085 ,
        -1.0549351 , -0.824805  ,  0.7344922 ],
       [ 0.3604965 ,  0.17747404, -1.1532713 , -0.52309954, -1.781787  ,
        -0.78317094, -0.9764233 ,  0.04816768,  0.7893254 ,  0.13798101,
         0.08532403, -0.15629251,  0.03805476, -0.50411135, -0.10637296,
         0.16995206,  1.2074281 ,  0.88629156],
       [ 0.7781441 , -1.7945913 ,  0.81046236, -0.77688366, -0.22636512,
        -0.2568624 ,  0.32405683, -0.64551955,  1.0603334 , -0.3441465 ,
        -0.99676   , -1.4548975 ,  0.7019724 , -1.0750128 , -0.02421461,
   